In [326]:
import pandas as pd
import statsmodels.api as sm 
import yfinance as yf
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import statistics
import seaborn as sns
import numpy as np
pd.set_option('display.max_rows', 10)

# Ex. 4.2 Student Performance and School Size

y = math10

In [327]:
# y = math10
ex_1 = pd.read_csv("Meap93.csv")
pairf_1 = ex_1.drop(['math10'], axis = 1)
y_1 = ex_1['math10']
ex_1.head()

,lnchprg,enroll,staff,expend,salary,benefits,droprate,gradrate,math10,sci11,totcomp,ltotcomp,lexpend,lenroll,lstaff,bensal,lsalary
0,1.4,1862,112.599999,5765,37498,7420,2.9,89.199997,56.400002,67.900002,44918,10.712594,8.659560,7.529407,4.723842,0.197877,10.532043
1,2.3,11355,101.199997,6601,48722,10370,1.3,91.400002,42.700001,65.300003,59092,10.986851,8.794976,9.337414,4.617099,0.212840,10.793886
2,2.7,7685,114.000000,6834,44541,7313,3.5,91.400002,43.799999,54.299999,51854,10.856188,8.829665,8.947025,4.736198,0.164186,10.704165
3,3.4,1148,85.400002,3586,31566,5989,3.6,86.599998,25.299999,60.000000,37555,10.533562,8.184793,7.045776,4.447346,0.189729,10.359836
4,3.4,1572,96.099998,3847,29781,5545,0.0,100.000000,15.300000,65.800003,35326,10.472375,8.255049,7.360104,4.565389,0.186193,10.301626


## (1) R square 

#### 刪掉log
樣本裡，有五個變數enroll, staff, expend, salary, totcomp 另外有產生log的變數，透過兩兩比較r squared決定要刪除哪個變數。

例如 : enroll 跟 log(enroll) 比較，enroll r squared比較高，所以保留 enroll

In [328]:
# 依據 : 哪個 R squared 高
# judge_log(該題所有變數的df , y的名稱字串, 有重複的變數的陣列 )
def judge_log(df, y='math10', log_x = ['enroll','staff','expend','salary','totcomp']):
    global throw
    print('R squared 比較高的:')
    throw = []  

    for col in log_x:
        x_model = sm.OLS(df[y], sm.add_constant(df[col])).fit()
        log_model = sm.OLS(df[y], sm.add_constant(df['l'+ col])).fit()
        if x_model.rsquared > log_model.rsquared:  
            print('  ', col, round(x_model.rsquared, 4))
            throw.append('l'+ col)
        elif x_model.rsquared < log_model.rsquared:  
            print('   log', col, round(log_model.rsquared, 4))
            throw.append(col)

    print('不要的重複變數: ', throw)
    pairf = df.drop([y]+ throw, axis = 1)
    return pairf

In [329]:
# 先放入有 log，重複的變數名，判斷要保留哪些變數
pairf_r = judge_log(ex_1, 'math10', ['enroll','staff','expend','salary','totcomp'])
print('\n自變數:', pairf_r.columns.tolist())
# 丟入模型
all_model = sm.OLS(y_1, sm.add_constant(pairf_r)).fit()
print(all_model.summary())

R squared 比較高的:
   enroll 0.0051
   log staff 0.0004
   expend 0.033
   log salary 0.0613
   log totcomp 0.0548
不要的重複變數:  ['lenroll', 'staff', 'lexpend', 'salary', 'totcomp']

自變數: ['lnchprg', 'enroll', 'expend', 'benefits', 'droprate', 'gradrate', 'sci11', 'ltotcomp', 'lstaff', 'bensal', 'lsalary']
                            OLS Regression Results                            
Dep. Variable:                 math10   R-squared:                       0.229
Model:                            OLS   Adj. R-squared:                  0.207
Method:                 Least Squares   F-statistic:                     10.69
Date:                Fri, 28 May 2021   Prob (F-statistic):           2.77e-17
Time:                        17:35:12   Log-Likelihood:                -1484.5
No. Observations:                 408   AIC:                             2993.
Df Residuals:                     396   BIC:                             3041.
Df Model:                          11                              

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


## (2) 利用 OVB、Multicollinearity、Quadratics 方式

### OVB

In [330]:
# 丟一組變數，利用變數平均計算出y，並與y平均比較
def ovb(y, df):
    print(statistics.mean(y))
    mean = []
    for i in df.columns:
        mean.append( statistics.mean(df[i]) )
    
    model = sm.OLS(y, sm.add_constant(df)).fit()
    regression_y = model.params['const']
    
    m = 0
    for i in df.columns:
        # print(regression_y,'+',model.params[i],'*',mean[m])
        regression_y = regression_y + model.params[i] * mean[m]
        m += 1
    print('估計出來的y = ', regression_y)
    return abs(regression_y - statistics.mean(y))

In [331]:
# 丟一組變數，輪流刪掉一個變數並跑迴圈，找出 OVB 最大的
def find_ovb_max(df):
    gap = []
    for i in df.columns:
        no_1 = df.drop([i], axis = 1)
        gap.append( ovb(no_1) ) 
    print( df.columns[ gap.index(max(gap)) ], max(gap))

In [332]:
# 檢驗前面被丟掉的變數 ['lenroll', 'staff', 'lexpend', 'salary', 'totcomp']，是否會有偏誤
print('第1小題目的結果模型，與y mean差距:', ovb(y_1, pairf_r), '\n')
for i in throw:
    recov = pd.concat([pairf_r, ex_1[i]], axis = 1)
    print('有自變數', i, '的 與y mean差距:', ovb(y_1, recov), '\n')

24.106862719487744
估計出來的y =  24.106862719483615
第1小題目的結果模型，與y mean差距: 4.128253294766182e-12 

24.106862719487744
估計出來的y =  24.106862719477213
有自變數 lenroll 的 與y mean差距: 1.0530243343964685e-11 

24.106862719487744
估計出來的y =  24.106862719484837
有自變數 staff 的 與y mean差距: 2.9061197892588098e-12 

24.106862719487744

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)



估計出來的y =  24.106862719476
有自變數 lexpend 的 與y mean差距: 1.1745271422114456e-11 

24.106862719487744
估計出來的y =  24.10686271948109
有自變數 salary 的 與y mean差距: 6.654232720393338e-12 

24.106862719487744
估計出來的y =  24.106862719483768
有自變數 totcomp 的 與y mean差距: 3.9754866065777605e-12 



差距都很小，沒有OVB

### Multicollinearity 共線性

In [333]:
# 計算所有變數的 vif ，並 print
def vif(df):
    cols = df.columns
    for i in cols:
        model = sm.OLS(df[i], sm.add_constant( df.drop([i], axis = 1)) ).fit()
        r = model.rsquared
        vif = 1./(1.- r)
        print(i, vif)

In [334]:
# 跑所有變數的VIF，並把VIF最大的變數剔除，並迴圈再跑，直到沒有變數VIF > 10
def vif_del(df):
    for m in range(10):
        a = []
        v = False
        cols = df.columns
        print('第', m, '次:')
        for i in cols:
            model = sm.OLS(df[i], sm.add_constant( df.drop([i], axis = 1)) ).fit()
            r = model.rsquared
            vif = 1./(1.- r)
            a.append(vif)
            if vif > 10:
                print(i, vif)     
                v = True
        if v == True:
            max_col = cols[a.index(max(a))] 
            df = df.drop([max_col], axis = 1)
            print('去除VIF最大的變數:', max_col, '\n')
        else:            
            print('沒有VIF > 10的變數')
            break;            
            
    return df

In [335]:
# 跑所有變數的VIF，並把VIF最大的變數剔除，並迴圈再跑，直到沒有變數VIF > 10
pairf_vif = vif_del(pairf_r)
pairf_vif

第 0 次:
expend 24.185391411125966
benefits 59.95908321912475
ltotcomp 11289.60522360858
lstaff 15.546695875046527
bensal 351.8945806393918
lsalary 11035.78285262456
去除VIF最大的變數: ltotcomp 

第 1 次:
expend 24.17449696491421
benefits 35.52428030350083
lstaff 15.546672966086039
bensal 25.286737995224474
lsalary 35.321628919419304
去除VIF最大的變數: benefits 

第 2 次:
expend 23.747649161578618
lstaff 15.294746510490477
lsalary 21.65728437580052
去除VIF最大的變數: expend 

第 3 次:
沒有VIF > 10的變數


,lnchprg,enroll,droprate,gradrate,sci11,lstaff,bensal,lsalary
0,1.400000,1862,2.900000,89.199997,67.900002,4.723842,0.197877,10.532043
1,2.300000,11355,1.300000,91.400002,65.300003,4.617099,0.212840,10.793886
2,2.700000,7685,3.500000,91.400002,54.299999,4.736198,0.164186,10.704165
3,3.400000,1148,3.600000,86.599998,60.000000,4.447346,0.189729,10.359836
4,3.400000,1572,0.000000,100.000000,65.800003,4.565389,0.186193,10.301626
...,...,...,...,...,...,...,...,...
403,60.900002,1689,35.799999,23.500000,11.200000,4.361824,0.226204,10.641991
404,61.099998,2631,17.500000,61.099998,22.700001,4.612146,0.218589,10.463675
405,73.800003,4024,8.700000,83.300003,26.799999,4.445002,0.190342,10.394458
406,74.400002,850,1.600000,91.900002,9.700000,4.807294,0.222972,10.282267


In [336]:
# 相關係數
def corr(pairf):
    for i in pairf.corr().columns:
        mul_list = pairf.corr().loc[pairf.corr()[i] > 0.7].index.tolist()
        mul_list = mul_list + pairf.corr().loc[pairf.corr()[i] < -0.7].index.tolist()
        for n in mul_list:
            if i != n:
                print(i , n, pairf.corr()[i][n])
corr(pairf_vif)

droprate gradrate -0.7185149935849265
gradrate droprate -0.7185149935849265


雖然VIF已篩選過，但 仍存在高度負相關的變數 : droprate & gradrate

droprate & gradrate 需二擇一，比較 R squared

In [337]:
no_gra_model = sm.OLS(y_1, sm.add_constant(pairf_vif.drop(['gradrate'], axis = 1))).fit()
no_dro_model = sm.OLS(y_1, sm.add_constant(pairf_vif.drop(['droprate'], axis = 1))).fit()
print('若去掉gradrate的rsquared:', round(no_gra_model.rsquared,6), '\n去掉droprate的rsquared:', round(no_dro_model.rsquared,6))

若去掉gradrate的rsquared: 0.198219 
去掉droprate的rsquared: 0.205021


In [338]:
#結論 : 去掉droprate的rsquared比較高 => 故保留 gradrate，去掉 droprate
pairf_vif_2 = pairf_vif.drop(['droprate'], axis = 1)
print('自變數:', pairf_vif_2.columns.tolist())
vif_model = sm.OLS(y_1, sm.add_constant(pairf_vif_2)).fit()
print(vif_model.summary())

自變數: ['lnchprg', 'enroll', 'gradrate', 'sci11', 'lstaff', 'bensal', 'lsalary']
                            OLS Regression Results                            
Dep. Variable:                 math10   R-squared:                       0.205
Model:                            OLS   Adj. R-squared:                  0.191
Method:                 Least Squares   F-statistic:                     14.74
Date:                Fri, 28 May 2021   Prob (F-statistic):           4.09e-17
Time:                        17:35:12   Log-Likelihood:                -1490.7
No. Observations:                 408   AIC:                             2997.
Df Residuals:                     400   BIC:                             3030.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

### Quadratics 尋找高次項

In [339]:
# 2次項
def quad(y, df):
    for i in df.columns:   
        a = df[i] ** 2
        df1 = pd.concat([df, a], axis = 1)
        model = sm.OLS(y, sm.add_constant(df1)).fit()
        print(model.rsquared)

# 2~5次項        
def power(y, df):
    model = sm.OLS(y, sm.add_constant(df)).fit()
    print('原始:', model.rsquared, '\nr2增加 > 5%:')
    for i in df.columns:  
        for n in range(1, 6):
            a = df[i] ** n
            df1 = pd.concat([df, a], axis = 1)
            model_1 = sm.OLS(y, sm.add_constant(df1)).fit()
            perc = (model_1.rsquared - model.rsquared) / model.rsquared
            if perc > 0.05:
                print(i, n, model_1.rsquared)

In [340]:
# 用第1小題結果，跑看看分別加入各變數的2~5次項時，r2有沒有增加 > 5% 幅度
power(y_1, pairf_r)

原始: 0.22890279626067556 
r2增加 > 5%:
expend 2 0.24218761636088904
expend 3 0.24059962497515408


C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


droprate 2 0.2550947220959945
droprate 3 0.24832737274622918
droprate 4 0.2456624977916021
droprate 5 0.24463327359549525
lstaff 2 0.2403998373661932
lstaff 3 0.24044221771832475
lstaff 4 0.24049062452387415
lstaff 5 0.24054477756990844


In [341]:
# 將上面的高次項，合併加入透過 VIF 算出來的變數裡
a = pd.DataFrame(pairf_r['expend'].values ** 2, columns = ['expend_2'])
b = pd.DataFrame(pairf_r['droprate'].values ** 2, columns = ['droprate_2'])
c =  pd.DataFrame(pairf_r['lstaff'].values ** 5, columns = ['lstaff_5'])
pairf_quad = pd.concat([pairf_vif_2, a, b, c], axis = 1)
pairf_quad

,lnchprg,enroll,gradrate,sci11,lstaff,bensal,lsalary,expend_2,droprate_2,lstaff_5
0,1.400000,1862,89.199997,67.900002,4.723842,0.197877,10.532043,33235225,8.410001,2352.213092
1,2.300000,11355,91.400002,65.300003,4.617099,0.212840,10.793886,43573201,1.690000,2098.194981
2,2.700000,7685,91.400002,54.299999,4.736198,0.164186,10.704165,46703556,12.250000,2383.139391
3,3.400000,1148,86.599998,60.000000,4.447346,0.189729,10.359836,12859396,12.959999,1739.821507
4,3.400000,1572,100.000000,65.800003,4.565389,0.186193,10.301626,14799409,0.000000,1983.302752
...,...,...,...,...,...,...,...,...,...,...
403,60.900002,1689,23.500000,11.200000,4.361824,0.226204,10.641991,28944400,1281.639946,1578.849401
404,61.099998,2631,61.099998,22.700001,4.612146,0.218589,10.463675,25887744,306.250000,2086.965101
405,73.800003,4024,83.300003,26.799999,4.445002,0.190342,10.394458,15539364,75.689997,1735.240235
406,74.400002,850,91.900002,9.700000,4.807294,0.222972,10.282267,25290841,2.560000,2567.459386


In [342]:
# 再用一次 VIF 跑看看，去除 VIF>10 的變數
vif_del(pairf_quad)

第 0 次:
lstaff 126.55335010132426
lsalary 13.613980155621528
expend_2 15.48692414558779
lstaff_5 137.4313696809124
去除VIF最大的變數: lstaff_5 

第 1 次:
lsalary 13.024795681409795
expend_2 14.079536381150422
去除VIF最大的變數: expend_2 

第 2 次:
沒有VIF > 10的變數


,lnchprg,enroll,gradrate,sci11,lstaff,bensal,lsalary,droprate_2
0,1.400000,1862,89.199997,67.900002,4.723842,0.197877,10.532043,8.410001
1,2.300000,11355,91.400002,65.300003,4.617099,0.212840,10.793886,1.690000
2,2.700000,7685,91.400002,54.299999,4.736198,0.164186,10.704165,12.250000
3,3.400000,1148,86.599998,60.000000,4.447346,0.189729,10.359836,12.959999
4,3.400000,1572,100.000000,65.800003,4.565389,0.186193,10.301626,0.000000
...,...,...,...,...,...,...,...,...
403,60.900002,1689,23.500000,11.200000,4.361824,0.226204,10.641991,1281.639946
404,61.099998,2631,61.099998,22.700001,4.612146,0.218589,10.463675,306.250000
405,73.800003,4024,83.300003,26.799999,4.445002,0.190342,10.394458,75.689997
406,74.400002,850,91.900002,9.700000,4.807294,0.222972,10.282267,2.560000


In [343]:
# 最終模型
pairf_quad_2 = pairf_quad.drop(['lstaff_5', 'expend_2'], axis = 1)
quad_model = sm.OLS(y_1, sm.add_constant(pairf_quad_2)).fit()
print(quad_model.summary())

                            OLS Regression Results                            
Dep. Variable:                 math10   R-squared:                       0.208
Model:                            OLS   Adj. R-squared:                  0.192
Method:                 Least Squares   F-statistic:                     13.10
Date:                Fri, 28 May 2021   Prob (F-statistic):           8.04e-17
Time:                        17:35:13   Log-Likelihood:                -1490.0
No. Observations:                 408   AIC:                             2998.
Df Residuals:                     399   BIC:                             3034.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -117.6721     49.953     -2.356      0.0

## (3) Stepwise regression 

### 向前法

In [344]:
def forward_regression(X, y, threshold_in, verbose=False):
    initial_list = []
    included = list(initial_list)
    while True:
        changed = False
        excluded = list(set(X.columns)-set(included)) #一個一個變數列入
        new_pval = pd.Series(index=excluded, dtype='float64')
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]  #每一個變數算p-value
        best_pval = new_pval.min() 
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin() #找最小值
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))
        if not changed:
            break
    return included

def backward_regression(X, y, threshold_out, verbose=False):
    included=list(X.columns)
    while True:
        changed=False
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit() #全部變數放入迴歸模型
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:] #計算p-value
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [345]:
result_1 = forward_regression(pairf_1, y_1, 0.05)

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

In [346]:
# 應變數'math10'，自變數 ['lnchprg', 'gradrate', 'expend', 'lexpend']
print(result_1)
model = sm.OLS(y_1, sm.add_constant(pairf_1[result_1])).fit()
print(model.summary())

['lnchprg', 'gradrate', 'expend', 'lexpend']
                            OLS Regression Results                            
Dep. Variable:                 math10   R-squared:                       0.208
Model:                            OLS   Adj. R-squared:                  0.200
Method:                 Least Squares   F-statistic:                     26.47
Date:                Fri, 28 May 2021   Prob (F-statistic):           1.64e-19
Time:                        17:35:14   Log-Likelihood:                -1489.9
No. Observations:                 408   AIC:                             2990.
Df Residuals:                     403   BIC:                             3010.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const  

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


### 向後法

In [347]:
result_1 = backward_regression(pairf_1, y_1, 0.01)  

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

In [348]:
print('自變數', result_1)
model = sm.OLS(y_1, sm.add_constant(ex_1[result_1])).fit()
print(model.summary())

自變數 ['lnchprg', 'enroll', 'expend', 'benefits', 'ltotcomp', 'lexpend', 'lenroll', 'lsalary']
                            OLS Regression Results                            
Dep. Variable:                 math10   R-squared:                       0.248
Model:                            OLS   Adj. R-squared:                  0.233
Method:                 Least Squares   F-statistic:                     16.48
Date:                Fri, 28 May 2021   Prob (F-statistic):           3.99e-21
Time:                        17:35:14   Log-Likelihood:                -1479.3
No. Observations:                 408   AIC:                             2977.
Df Residuals:                     399   BIC:                             3013.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


### 逐步挑選(Stepwise)-向前與向後混和

In [349]:
def stepwise_selection(X, y, initial_list=[], threshold_in=0.05, 
                       threshold_out=0.05, verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    initial_list = []
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded,dtype='float64')
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        #backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [350]:
result_1 = stepwise_selection(pairf_1, y_1)

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

Add  lnchprg                        with p-value 2.74661e-18


C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

Add  gradrate                       with p-value 0.00700876


C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

Add  expend                         with p-value 0.0146886


C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

Add  lexpend                        with p-value 0.0221231


C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

In [351]:
# 應變數'math10'，自變數['lnchprg', 'gradrate', 'expend', 'lexpend']
print(result_1)
model = sm.OLS(y_1, sm.add_constant(ex_1[result_1])).fit()
print(model.summary())

['lnchprg', 'gradrate', 'expend', 'lexpend']
                            OLS Regression Results                            
Dep. Variable:                 math10   R-squared:                       0.208
Model:                            OLS   Adj. R-squared:                  0.200
Method:                 Least Squares   F-statistic:                     26.47
Date:                Fri, 28 May 2021   Prob (F-statistic):           1.64e-19
Time:                        17:35:15   Log-Likelihood:                -1489.9
No. Observations:                 408   AIC:                             2990.
Df Residuals:                     403   BIC:                             3010.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const  

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


# Ex. 4.5 Housing Prices and Air Pollution

1. price 是 log(price) ，題目y採用 lprice，所以刪除 price

In [352]:
ex_2 = pd.read_csv("Hprice2.csv")
ex_2 = ex_2.drop(['price'], axis = 1)
pairf_2 = ex_2.drop(['lprice'], axis = 1)
y_2 = ex_2['lprice']
ex_2.head()

,crime,nox,rooms,dist,radial,proptax,stratio,lowstat,lprice,lnox,lproptax
0,0.006,5.38,6.57,4.09,1,29.600000,15.300000,4.98,10.085809,1.682688,5.690360
1,0.027,4.69,6.42,4.97,2,24.200001,17.799999,9.14,9.980402,1.545433,5.488938
2,0.027,4.69,7.18,4.97,2,24.200001,17.799999,4.03,10.454495,1.545433,5.488938
3,0.032,4.58,7.00,6.06,3,22.200001,18.700001,2.94,10.416311,1.521699,5.402678
4,0.069,4.58,7.15,6.06,3,22.200001,18.700001,5.33,10.496787,1.521699,5.402678


## (1) R squared

In [353]:
# 先放入有 log，重複的變數名，判斷要保留哪些變數
pairf_r = judge_log(ex_2, 'lprice', ['nox','proptax'])
print('\n自變數:', pairf_r.columns.tolist())
# 丟入模型
model = sm.OLS(y_2, sm.add_constant(pairf_r)).fit()
print(model.summary())

R squared 比較高的:
   log nox 0.2636
   proptax 0.3132
不要的重複變數:  ['nox', 'lproptax']

自變數: ['crime', 'rooms', 'dist', 'radial', 'proptax', 'stratio', 'lowstat', 'lnox']
                            OLS Regression Results                            
Dep. Variable:                 lprice   R-squared:                       0.766
Model:                            OLS   Adj. R-squared:                  0.763
Method:                 Least Squares   F-statistic:                     203.8
Date:                Fri, 28 May 2021   Prob (F-statistic):          1.35e-151
Time:                        17:35:15   Log-Likelihood:                 102.51
No. Observations:                 506   AIC:                            -187.0
Df Residuals:                     497   BIC:                            -149.0
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t 

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


## (2) 利用 OVB、Multicollinearity、Quadratics 方式

### OVB

In [354]:
# 檢驗前面被丟掉的變數，是否會有偏誤
print('第1小題目的結果模型，與y mean差距:', ovb(y_2, pairf_r), '\n')
for i in throw:
    recov = pd.concat([pairf_r, ex_2[i]], axis = 1)
    print('有自變數', i, '的 與y mean差距:', ovb(y_2, recov), '\n')

# 差距都很小，沒有OVB

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


9.941057111033597
估計出來的y =  9.941057111033514
第1小題目的結果模型，與y mean差距: 8.348877145181177e-14 

9.941057111033597
估計出來的y =  9.941057111033508
有自變數 nox 的 與y mean差距: 8.881784197001252e-14 

9.941057111033597
估計出來的y =  9.941057111033501
有自變數 lproptax 的 與y mean差距: 9.592326932761353e-14 



### Multicollinearity 共線性

In [355]:
# 跑所有變數的VIF，並把VIF最大的變數剔除，並迴圈再跑，直到沒有變數VIF > 10
pairf_vif = vif_del(pairf_r)
pairf_vif

第 0 次:
沒有VIF > 10的變數


,crime,rooms,dist,radial,proptax,stratio,lowstat,lnox
0,0.006,6.57,4.09,1,29.600000,15.300000,4.98,1.682688
1,0.027,6.42,4.97,2,24.200001,17.799999,9.14,1.545433
2,0.027,7.18,4.97,2,24.200001,17.799999,4.03,1.545433
3,0.032,7.00,6.06,3,22.200001,18.700001,2.94,1.521699
4,0.069,7.15,6.06,3,22.200001,18.700001,5.33,1.521699
...,...,...,...,...,...,...,...,...
501,0.063,6.59,2.48,1,27.299999,21.000000,9.67,1.745715
502,0.045,6.12,2.29,1,27.299999,21.000000,9.08,1.745715
503,0.061,6.98,2.17,1,27.299999,21.000000,5.64,1.745715
504,0.110,6.79,2.39,1,27.299999,21.000000,6.48,1.745715


### Quadratics 尋找高次項

In [356]:
# 用第1小題結果，跑看看分別加入各變數的2~5次項時，r2有沒有增加 > 5% 幅度
power(y_2, pairf_r)

原始: 0.7664215237187069 
r2增加 > 5%:


沒有高次項

In [357]:
model = sm.OLS(y_2, sm.add_constant(pairf_vif)).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 lprice   R-squared:                       0.766
Model:                            OLS   Adj. R-squared:                  0.763
Method:                 Least Squares   F-statistic:                     203.8
Date:                Fri, 28 May 2021   Prob (F-statistic):          1.35e-151
Time:                        17:35:15   Log-Likelihood:                 102.51
No. Observations:                 506   AIC:                            -187.0
Df Residuals:                     497   BIC:                            -149.0
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         11.5388      0.250     46.158      0.0

## (3) Stepwise regression

### 向前法

In [358]:
result_2 = forward_regression(pairf_2, y_2, 0.05)
print(result_2)

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

['lowstat', 'stratio', 'crime', 'rooms', 'dist', 'nox', 'radial', 'lproptax']


C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

In [359]:
model = sm.OLS(y_2, sm.add_constant(pairf_2[result_2])).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 lprice   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     205.3
Date:                Fri, 28 May 2021   Prob (F-statistic):          3.49e-152
Time:                        17:35:16   Log-Likelihood:                 103.89
No. Observations:                 506   AIC:                            -189.8
Df Residuals:                     497   BIC:                            -151.7
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.3055      0.327     37.610      0.0

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


### 向後法

In [360]:
result_2 = backward_regression(pairf_2, y_2, 0.01)  
print('自變數', result_2)

自變數 ['crime', 'nox', 'rooms', 'dist', 'radial', 'stratio', 'lowstat', 'lproptax']


C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [361]:
model = sm.OLS(y_2, sm.add_constant(ex_2[result_2])).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 lprice   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     205.3
Date:                Fri, 28 May 2021   Prob (F-statistic):          3.49e-152
Time:                        17:35:16   Log-Likelihood:                 103.89
No. Observations:                 506   AIC:                            -189.8
Df Residuals:                     497   BIC:                            -151.7
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.3055      0.327     37.610      0.0

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


### 逐步挑選(Stepwise) - 向前與向後混和

In [362]:
result_2 = stepwise_selection(pairf_2, y_2)
print(result_2)

Add  lowstat                        with p-value 8.04215e-110


C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

Add  stratio                        with p-value 1.74359e-17


C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

Add  crime                          with p-value 6.69191e-11


...

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

Add  radial                         with p-value 0.0168947


C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


Add  lproptax                       with p-value 3.31765e-06
['lowstat', 'stratio', 'crime', 'rooms', 'dist', 'nox', 'radial', 'lproptax']


C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [363]:
model = sm.OLS(y_2, sm.add_constant(ex_2[result_2])).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 lprice   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     205.3
Date:                Fri, 28 May 2021   Prob (F-statistic):          3.49e-152
Time:                        17:35:16   Log-Likelihood:                 103.89
No. Observations:                 506   AIC:                            -189.8
Df Residuals:                     497   BIC:                            -151.7
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.3055      0.327     37.610      0.0

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


# Ex. 4.8 Model of R&D Expenditures

1. lrd 是 log(rd) ，題目y採用 lrd，所以刪除 rd

In [364]:
ex_3 = pd.read_csv("RDCHEM.csv")
ex_3 = ex_3.drop(['rd'], axis = 1)
pairf_3 = ex_3.drop(['lrd'], axis = 1)
y_3 = ex_3['lrd']
ex_3.head()

,sales,profits,rdintens,profmarg,salessq,lsales,lrd
0,4570.200195,186.899994,9.421906,4.089536,2.088673e+07,8.427312,6.065180
1,2830.000000,467.000000,2.084806,16.501766,8.008900e+06,7.948032,4.077538
2,596.799988,107.400001,3.937668,17.995979,3.561702e+05,6.391582,3.157000
3,133.600006,-4.300000,2.619760,-3.218563,1.784896e+04,4.894850,1.252763
4,42.000000,8.000000,4.047619,19.047619,1.764000e+03,3.737670,0.530628


## (1) R squared

In [365]:
# 先放入有 log，重複的變數名，判斷要保留哪些變數
pairf_r = judge_log(ex_3, 'lrd', ['sales'])
print('\n自變數:', pairf_r.columns.tolist())
# 丟入模型
model = sm.OLS(y_3, sm.add_constant(pairf_r)).fit()
print(model.summary())

R squared 比較高的:
   log sales 0.9098
不要的重複變數:  ['sales']

自變數: ['profits', 'rdintens', 'profmarg', 'salessq', 'lsales']
                            OLS Regression Results                            
Dep. Variable:                    lrd   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                     686.7
Date:                Fri, 28 May 2021   Prob (F-statistic):           9.77e-27
Time:                        17:35:16   Log-Likelihood:                 15.734
No. Observations:                  32   AIC:                            -19.47
Df Residuals:                      26   BIC:                            -10.67
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


## (2) 利用 OVB、Multicollinearity、Quadratics 方式

### OVB

In [366]:
# 檢驗前面被丟掉的變數，是否會有偏誤
print('第1小題目的結果模型，與y mean差距:', ovb(y_3, pairf_r), '\n')
for i in throw:
    recov = pd.concat([pairf_r, ex_3[i]], axis = 1)
    print('有自變數', i, '的 與y mean差距:', ovb(y_3, recov), '\n')

# 差距都很小，沒有OVB

3.602824965625
估計出來的y =  3.602824965624893
第1小題目的結果模型，與y mean差距: 1.0702549957386509e-13 

3.602824965625
估計出來的y =  3.602824965636414
有自變數 sales 的 與y mean差距: 1.141398087156631e-11 



C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


### Multicollinearity 共線性

In [367]:
# 跑所有變數的VIF，並把VIF最大的變數剔除，並迴圈再跑，直到沒有變數VIF > 10
pairf_vif = vif_del(pairf_r)
pairf_vif

第 0 次:
profits 14.654721227496479
salessq 11.148755929868964
去除VIF最大的變數: profits 

第 1 次:
沒有VIF > 10的變數


,rdintens,profmarg,salessq,lsales
0,9.421906,4.089536,2.088673e+07,8.427312
1,2.084806,16.501766,8.008900e+06,7.948032
2,3.937668,17.995979,3.561702e+05,6.391582
3,2.619760,-3.218563,1.784896e+04,4.894850
4,4.047619,19.047619,1.764000e+03,3.737670
...,...,...,...,...
27,1.214656,3.663879,2.522049e+05,6.218998
28,6.310198,11.083570,7.974976e+06,7.945909
29,1.026694,2.190281,8.538084e+04,5.677439
30,2.506233,8.214145,5.807964e+07,8.938663


### Quadratics 尋找高次項

In [368]:
# 用第1小題結果，跑看看分別加入各變數的2~5次項時，r2有沒有增加 > 5% 幅度
power(y_3, pairf_r)

原始: 0.9924841626871224 
r2增加 > 5%:


## (3) Stepwise regression

### 向前法

In [369]:
result_3 = forward_regression(pairf_3, y_3, 0.05)
print(result_3)

['lsales', 'rdintens']


C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

In [370]:
model = sm.OLS(y_3, sm.add_constant(pairf_3[result_3])).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    lrd   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     1586.
Date:                Fri, 28 May 2021   Prob (F-statistic):           2.38e-30
Time:                        17:35:17   Log-Likelihood:                 12.747
No. Observations:                  32   AIC:                            -19.49
Df Residuals:                      29   BIC:                            -15.10
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.4052      0.147    -29.938      0.0

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


### 向後法

In [371]:
result_3 = backward_regression(pairf_3, y_3, 0.01)  
print('自變數', result_3)

自變數 ['rdintens', 'lsales']


C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

In [372]:
model = sm.OLS(y_3, sm.add_constant(ex_3[result_3])).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    lrd   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     1586.
Date:                Fri, 28 May 2021   Prob (F-statistic):           2.38e-30
Time:                        17:35:17   Log-Likelihood:                 12.747
No. Observations:                  32   AIC:                            -19.49
Df Residuals:                      29   BIC:                            -15.10
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.4052      0.147    -29.938      0.0

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


### 逐步挑選(Stepwise) - 向前與向後混和

In [373]:
result_3 = stepwise_selection(pairf_3, y_3)
print(result_3)

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

Add  lsales                         with p-value 3.19558e-17
Add  rdintens                       with p-value 5.21584e-16
['lsales', 'rdintens']


C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

In [374]:
model = sm.OLS(y_3, sm.add_constant(ex_3[result_3])).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    lrd   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     1586.
Date:                Fri, 28 May 2021   Prob (F-statistic):           2.38e-30
Time:                        17:35:17   Log-Likelihood:                 12.747
No. Observations:                  32   AIC:                            -19.49
Df Residuals:                      29   BIC:                            -15.10
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.4052      0.147    -29.938      0.0

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


# Ex. 4.9 Parents' Education in a Birth Weight Equation

1. lbwght 是 log(bwght)，所以刪除
2. bwght跟bwghtlbs一樣，只是單位不同，所以刪除bwghtlbs
3. 刪除缺失值

In [375]:
ex_4 = pd.read_csv("Bwght.csv")
ex_4 = ex_4.dropna()          #刪掉缺失值
ex_4 = ex_4.reset_index()  
ex_4 = ex_4.drop(['index', 'lbwght', 'bwghtlbs'], axis = 1)
pairf_4 = ex_4.drop(['bwght'], axis = 1)
y_4 = ex_4['bwght']
ex_4#.head()

,faminc,cigtax,cigprice,bwght,fatheduc,motheduc,parity,male,white,cigs,packs,lfaminc
0,13.5,16.5,122.300003,109,12.0,12.0,1,1,1,0,0.0,2.602690
1,7.5,16.5,122.300003,133,6.0,12.0,2,1,0,0,0.0,2.014903
2,15.5,16.5,122.300003,126,12.0,12.0,2,1,0,0,0.0,2.740840
3,27.5,16.5,122.300003,134,14.0,12.0,2,1,1,0,0.0,3.314186
4,7.5,16.5,122.300003,118,12.0,14.0,6,1,0,0,0.0,2.014903
...,...,...,...,...,...,...,...,...,...,...,...,...
1186,12.5,30.0,138.300003,87,14.0,12.0,1,1,1,0,0.0,2.525729
1187,22.5,30.0,138.300003,135,16.0,16.0,3,0,1,0,0.0,3.113515
1188,27.5,30.0,138.300003,110,12.0,12.0,4,1,1,0,0.0,3.314186
1189,65.0,8.0,118.599999,135,18.0,16.0,2,0,1,0,0.0,4.174387


## (1) R squared

In [376]:
# 先放入有 log，重複的變數名，判斷要保留哪些變數
pairf_r = judge_log(ex_4, 'bwght', ['faminc'])
print('自變數:', pairf_r.columns.tolist())
# 丟入模型
model = sm.OLS(y_4, sm.add_constant(pairf_r)).fit()
print(model.summary())

R squared 比較高的:
   log faminc 0.0066
不要的重複變數:  ['faminc']
自變數: ['cigtax', 'cigprice', 'fatheduc', 'motheduc', 'parity', 'male', 'white', 'cigs', 'packs', 'lfaminc']
                            OLS Regression Results                            
Dep. Variable:                  bwght   R-squared:                       0.056
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     6.994
Date:                Fri, 28 May 2021   Prob (F-statistic):           9.97e-11
Time:                        17:35:17   Log-Likelihood:                -5231.5
No. Observations:                1191   AIC:                         1.048e+04
Df Residuals:                    1180   BIC:                         1.054e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t  

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


## (2) 利用 OVB、Multicollinearity、Quadratics 方式

### OVB

In [377]:
# 檢驗前面被丟掉的變數，是否會有偏誤
print('第1小題目的結果模型，與y mean差距:', ovb(y_4, pairf_r), '\n')
for i in throw:
    recov = pd.concat([pairf_r, ex_4[i]], axis = 1)
    print('有自變數', i, '的 與y mean差距:', ovb(y_4, recov), '\n')

# 差距都很小，沒有OVB

119.52980688497061
估計出來的y =  119.5298069310315
第1小題目的結果模型，與y mean差距: 4.606089021308435e-08 

119.52980688497061
估計出來的y =  119.5298067735011
有自變數 faminc 的 與y mean差距: 1.1146951806040306e-07 



C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


### Multicollinearity 共線性

In [378]:
# 跑所有變數的VIF，並把VIF最大的變數剔除，並迴圈再跑，直到沒有變數VIF > 10
pairf_vif = vif_del(pairf_r)
pairf_vif

第 0 次:


C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


cigs inf
packs inf
去除VIF最大的變數: cigs 

第 1 次:
沒有VIF > 10的變數


,cigtax,cigprice,fatheduc,motheduc,parity,male,white,packs,lfaminc
0,16.5,122.300003,12.0,12.0,1,1,1,0.0,2.602690
1,16.5,122.300003,6.0,12.0,2,1,0,0.0,2.014903
2,16.5,122.300003,12.0,12.0,2,1,0,0.0,2.740840
3,16.5,122.300003,14.0,12.0,2,1,1,0.0,3.314186
4,16.5,122.300003,12.0,14.0,6,1,0,0.0,2.014903
...,...,...,...,...,...,...,...,...,...
1186,30.0,138.300003,14.0,12.0,1,1,1,0.0,2.525729
1187,30.0,138.300003,16.0,16.0,3,0,1,0.0,3.113515
1188,30.0,138.300003,12.0,12.0,4,1,1,0.0,3.314186
1189,8.0,118.599999,18.0,16.0,2,0,1,0.0,4.174387


雖然VIF已篩選過，但 仍存在高度相關的變數 : cigtax cigprice

cigtax cigprice 需二擇一，比較 R squared

### Quadratics 尋找高次項

In [379]:
# 用第1小題結果，跑看看分別加入各變數的2~5次項時，r2有沒有增加 > 5% 幅度
power(y_4, pairf_r)

原始: 0.05595453256265159 
r2增加 > 5%:
cigtax 2 0.05951139452843279
cigtax 3 0.05971898125223085
cigtax 4 0.05966507774307561
cigtax 5 0.059128695125432174
motheduc 2 0.05920365501846225
motheduc 3 0.05951236207550081
motheduc 4 0.05973638214494792
motheduc 5 0.059887914933276054


In [380]:
# 將上面的高次項，合併加入透過 VIF 算出來的變數裡
a = pd.DataFrame(pairf_r['cigtax'].values ** 3, columns = ['cigtaxd_3'])
b = pd.DataFrame(pairf_r['motheduc'].values ** 5, columns = ['motheduc_5'])
pairf_quad = pd.concat([pairf_vif, a, b], axis = 1)
pairf_quad

,cigtax,cigprice,fatheduc,motheduc,parity,male,white,packs,lfaminc,cigtaxd_3,motheduc_5
0,16.5,122.300003,12.0,12.0,1,1,1,0.0,2.602690,4492.125,248832.0
1,16.5,122.300003,6.0,12.0,2,1,0,0.0,2.014903,4492.125,248832.0
2,16.5,122.300003,12.0,12.0,2,1,0,0.0,2.740840,4492.125,248832.0
3,16.5,122.300003,14.0,12.0,2,1,1,0.0,3.314186,4492.125,248832.0
4,16.5,122.300003,12.0,14.0,6,1,0,0.0,2.014903,4492.125,537824.0
...,...,...,...,...,...,...,...,...,...,...,...
1186,30.0,138.300003,14.0,12.0,1,1,1,0.0,2.525729,27000.000,248832.0
1187,30.0,138.300003,16.0,16.0,3,0,1,0.0,3.113515,27000.000,1048576.0
1188,30.0,138.300003,12.0,12.0,4,1,1,0.0,3.314186,27000.000,248832.0
1189,8.0,118.599999,18.0,16.0,2,0,1,0.0,4.174387,512.000,1048576.0


In [381]:
vif_del(pairf_quad)

第 0 次:
沒有VIF > 10的變數


,cigtax,cigprice,fatheduc,motheduc,parity,male,white,packs,lfaminc,cigtaxd_3,motheduc_5
0,16.5,122.300003,12.0,12.0,1,1,1,0.0,2.602690,4492.125,248832.0
1,16.5,122.300003,6.0,12.0,2,1,0,0.0,2.014903,4492.125,248832.0
2,16.5,122.300003,12.0,12.0,2,1,0,0.0,2.740840,4492.125,248832.0
3,16.5,122.300003,14.0,12.0,2,1,1,0.0,3.314186,4492.125,248832.0
4,16.5,122.300003,12.0,14.0,6,1,0,0.0,2.014903,4492.125,537824.0
...,...,...,...,...,...,...,...,...,...,...,...
1186,30.0,138.300003,14.0,12.0,1,1,1,0.0,2.525729,27000.000,248832.0
1187,30.0,138.300003,16.0,16.0,3,0,1,0.0,3.113515,27000.000,1048576.0
1188,30.0,138.300003,12.0,12.0,4,1,1,0.0,3.314186,27000.000,248832.0
1189,8.0,118.599999,18.0,16.0,2,0,1,0.0,4.174387,512.000,1048576.0


In [382]:
# 最終模型
print(pairf_quad.columns.tolist())
quad_model = sm.OLS(y_4, sm.add_constant(pairf_quad)).fit()
print(quad_model.summary())

['cigtax', 'cigprice', 'fatheduc', 'motheduc', 'parity', 'male', 'white', 'packs', 'lfaminc', 'cigtaxd_3', 'motheduc_5']
                            OLS Regression Results                            
Dep. Variable:                  bwght   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.055
Method:                 Least Squares   F-statistic:                     7.249
Date:                Fri, 28 May 2021   Prob (F-statistic):           4.58e-12
Time:                        17:35:18   Log-Likelihood:                -5226.8
No. Observations:                1191   AIC:                         1.048e+04
Df Residuals:                    1179   BIC:                         1.054e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------

## (3) Stepwise regression

### 向前法

In [383]:
result_4 = forward_regression(pairf_4, y_4, 0.05)
print(result_4)

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

['packs', 'white', 'male', 'parity']


C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

In [384]:
model = sm.OLS(y_4, sm.add_constant(pairf_4[result_4])).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  bwght   R-squared:                       0.050
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     15.71
Date:                Fri, 28 May 2021   Prob (F-statistic):           1.56e-12
Time:                        17:35:18   Log-Likelihood:                -5235.0
No. Observations:                1191   AIC:                         1.048e+04
Df Residuals:                    1186   BIC:                         1.051e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        111.0851      1.943     57.172      0.0

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


### 向後法

In [385]:
result_4 = backward_regression(pairf_4, y_4, 0.01)  
print('自變數', result_4)

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


自變數 ['parity', 'male', 'white', 'packs']


C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

In [386]:
model = sm.OLS(y_4, sm.add_constant(ex_4[result_4])).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  bwght   R-squared:                       0.050
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     15.71
Date:                Fri, 28 May 2021   Prob (F-statistic):           1.56e-12
Time:                        17:35:19   Log-Likelihood:                -5235.0
No. Observations:                1191   AIC:                         1.048e+04
Df Residuals:                    1186   BIC:                         1.051e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        111.0851      1.943     57.172      0.0

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


### 逐步挑選(Stepwise) - 向前與向後混和

In [387]:
result_4 = stepwise_selection(pairf_4, y_4)
print(result_4)

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

Add  packs                          with p-value 1.11126e-08


C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

Add  white                          with p-value 0.00162568


C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

Add  male                           with p-value 0.0014023


C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\sandy\Anaconda3\envs\Quantative\lib

Add  parity                         with p-value 0.00345897
['packs', 'white', 'male', 'parity']


In [388]:
model = sm.OLS(y_4, sm.add_constant(ex_4[result_4])).fit()
print(model.summary())

C:\Users\sandy\Anaconda3\envs\Quantative\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


                            OLS Regression Results                            
Dep. Variable:                  bwght   R-squared:                       0.050
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     15.71
Date:                Fri, 28 May 2021   Prob (F-statistic):           1.56e-12
Time:                        17:35:19   Log-Likelihood:                -5235.0
No. Observations:                1191   AIC:                         1.048e+04
Df Residuals:                    1186   BIC:                         1.051e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        111.0851      1.943     57.172      0.0